In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea

In [3]:
# import data 
df = pd.read_csv('/Users/arthurrastelli/Desktop/DS/Assignment/Assignment.csv')
df.head()

,AGEPH,CODPOSS,duree,lnexpo,nbrtotc,nbrtotan,chargtot,agecar,sexp,fuelc,split,usec,fleetc,sportc,coverp,powerc
0,64,1000,1.000000,0.000000,0,0.000000,0.000000,2-5,Female,Petrol,Once,Private,No,No,MTPL+,66-110
1,28,1000,0.046575,-3.066684,1,21.470588,155.974606,6-10,Female,Petrol,Twice,Private,No,No,MTPL,66-110
2,58,1000,0.402740,-0.909465,0,0.000000,0.000000,>10,Female,Petrol,Thrice,Private,No,No,MTPL,<66
3,37,1030,0.169863,-1.772763,0,0.000000,0.000000,2-5,Female,Petrol,Once,Professional,No,No,MTPL+++,66-110
4,29,1030,1.000000,0.000000,0,0.000000,0.000000,6-10,Female,Petrol,Once,Private,No,No,MTPL+,<66


In [24]:
print(df.iloc[:, 0].dtypes)
print(df.iloc[:, 1].dtypes)
print(df.iloc[:, 2].dtypes)
print(df.iloc[:, 3].dtypes)
print(df.iloc[:, 4].dtypes)
print(df.iloc[:, 5].dtypes)
print(df.iloc[:, 6].dtypes)
print(df.iloc[:, 7].dtypes)
print(df.iloc[:, 8].dtypes)
print(df.iloc[:, 9].dtypes)
print(df.iloc[:, 10].dtypes)
print(df.iloc[:, 12].dtypes)
print(df.iloc[:, 13].dtypes)
print(df.iloc[:, 14].dtypes)
print(df.iloc[:, 15].dtypes)

int64
int64
float64
float64
int64
float64
float64
object
object
object
object
object
object
object
object


In [ ]:
# Histograms, ... 

In [ ]:
# set up dummy variabels? 

In [ ]:
# Check which variables are 'correlated' or need to have a interaction effect